
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.4 - alpha preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Pre-sail checklist - Python API for Ocean
With the Ocean Protocol components running, test the Squid API (Python API).
Instantiate the API with your selected `config.ini` file, or use the default for this environment.

## Import the API, `squid-py`, and a simple utilities library `mantaray_utilities`.

In [11]:
# Standard imports
import logging

# Import mantaray and the Ocean API (squid)
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
manta_utils.logging.logger.setLevel('CRITICAL')

print("squid-py Ocean API version:", squid_py.__version__)

squid-py Ocean API version: 0.4.3


In order to manage the different environments Mantaray runs in, we have a series of environment variables
which are used in the utilities library to resolve paths and keep behavior consistent. In the JupyterHub
deployment, all of this is taken care of for you.

In [12]:
# Get the configuration file path for this environment
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))

CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))

50 - <ipython-input-12-e7efd7b9720d> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-12-e7efd7b9720d> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini


## Connect to Ocean Protocal with the configuration file

In [13]:
# Load the configuration
configuration = Config(CONFIG_INI_PATH)
print("Configuration loaded. Will connect to a node at: ", configuration.keeper_url)

Configuration loaded. Will connect to a node at:  https://nile.dev-ocean.com


Feel free to inspect the `configuration` object.

From the configuration, instantiate the Ocean object, the primary interface to Ocean Protocol.

In [14]:
# Instantiate Ocean
ocn = Ocean(configuration)

The following cell will print some summary information of the Ocean connection.
TODO: add pretty printing of the connection

In [15]:
print("***OCEAN***")
print("{} accounts".format(len(ocn.accounts._accounts)))
for i, account in enumerate(ocn.accounts._accounts):
    print(i, account.address)

***OCEAN***
27 accounts
0 0x0207cb2f99EB2e005893d6108E2633641Ca9cC3e
1 0x064789569D09b4d40b54383d84A25A840E5D67aD
2 0x06C0035fE67Cce2B8862D63Dc315D8C8c72207cA
3 0x07aCfF76eB4A10c92B89965B0e174B35d887cB0b
4 0x0A7a04DEdFE9A4859bcEdD2F21dcA8110499bed9
5 0x1322A6ef2c560107733bFc622Fe556961Cb430a5
6 0x1549193cc522A1716be4886879cD5f0964A3073C
7 0x15C8249D657ad28daa56662ad4ee18dA4377d1a4
8 0x170ff54d7bbD527B7bdF04770845bd727a531d9c
9 0x1769AAf66658DC2D7782796ac2fF350d4ecf96Cd
10 0x17d25A173C9914ee09221EA8E490D296121e51F2
11 0x291456dbD9119Ed4e4FE261246F5985ebF4e7Ae9
12 0x2c918a0EF92f0b7113c8c4a966CC4f22AfA5652f
13 0x2DFeEe7Ac1a126AF0d254235D0e21F43d5c9B103
14 0x2E92872deE4Cd4f5d5544C257cAa2A66710bE73D
15 0x33333759e516dD51b533777b733a40c2EbD4869D
16 0x364BA547Fb1a96881D59CC193641DDB4D891a6E2
17 0x3C5bdEBBB588f6d7ebBbC74b6Ed56f3b9dF2f7e2
18 0x3F367a69C2559a4B90044eA6b6dfF7E0fE4a52E6
19 0x413c9BA0A05B8A600899B41b0c62dd661e689354
20 0x43E6b79B696b1d8E21d5A23118A1BF1b565C3831
21 0x442aCf84E05459e

## Alternatively, connect to Ocean with a configuration as dictionary
The configuration of the client (mantaray) can be inspected in the below code cells. The following configuration
is set for local testing.

In [16]:
config_dict = {
    'keeper-contracts': {
        # Point to an Ethereum RPC client. Note that Squid learns the name of the network to work with from this client.
        'keeper.url': 'http://localhost:8545',
        # Specify the keeper contracts artifacts folder (has the smart contracts definitions json files). When you
        # install the package, the artifacts are automatically picked up from the `keeper-contracts` Python
        # dependency unless you are using a local ethereum network.
        'keeper.path': 'artifacts',
        'secret_store.url': 'http://localhost:12001',
        'parity.url': 'http://localhost:8545',
        'parity.address': '',
        'parity.password': '',

    },
    'resources': {
        # aquarius is the metadata store. It stores the assets DDO/DID-document
        'aquarius.url': 'http://localhost:5000',
        # Brizo is the publisher's agent. It serves purchase and requests for both data access and compute services
        'brizo.url': 'http://localhost:8030',
        # points to the local database file used for storing temporary information (for instance, pending service agreements).
        'storage.path': 'squid_py.db',
        # Where to store downloaded asset files
        'downloads.path': 'consume-downloads'
    }
}

In [9]:
# You may modify the dictionary object and uncomment the next cell to test

```
# Instantiate Ocean from the above dictionary
configuration = Config(filename=None, options_dict=config_dict)
ocn = Ocean(configuration)
print("***OCEAN***")
print("{} accounts".format(len(ocn.accounts._accounts)))
for i, account in enumerate(ocn.accounts._accounts):
    print(i, account.address)
```